In [1]:
#2018-07-30 09:09:48.209664
%load_ext metapack.jupyter.magic

In [2]:
CACHE_DIR='/Users/eric/Library/Application Support/metapack/'
RESOURCE_NAME='tides_river_rain'
RESOLVED_URL='file:///Users/eric/proj/virt-proj/water/water-datasets/sandiegodata.org-water_quality/notebooks/Combine.ipynb#comb'
WORKING_DIR='/Users/eric/proj/virt-proj/water/water-datasets/sandiegodata.org-water_quality'
METATAB_DOC='metapack+file:///Users/eric/proj/virt-proj/water/water-datasets/sandiegodata.org-water_quality/metadata.csv'
METATAB_WORKING_DIR='/Users/eric/proj/virt-proj/water/water-datasets/sandiegodata.org-water_quality'
METATAB_PACKAGE='metapack+file:///Users/eric/proj/virt-proj/water/water-datasets/sandiegodata.org-water_quality/'
name='tides_river_rain'
description='Combined max, min tides, San Diego river flow and daily precipitation'
url='notebooks/Combine.ipynb#comb'

In [3]:
METAPACK_BUILDING=True

In [4]:
import matplotlib.pyplot as plt 
import metapack as mp
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline

from metapack.appurl import SearchUrl 
SearchUrl.initialize()

In [5]:
pkg = mp.jupyter.open_source_package()
pkg

In [6]:
r =  pkg.reference('la_jolla_tide')
r

Header,Type,Description
date_time,datetime,
water_level,number,"Water level, relative to datum, in feet"
type,string,"HH Daily high, H High, LL Daily Low, L Low"
I,integer,
L,integer,


In [7]:
water_levels = r.read_csv(parse_dates=True)
wl_by_day = water_levels.set_index(['date_time','type']).water_level.unstack(1).resample('1d').max()
wl_by_day.columns = ['wl_high_1', 'wl_high_2', 'wl_low_1','wl_low_2']
wl_by_day['wl_max'] = np.maximum(wl_by_day.wl_high_1, wl_by_day.wl_high_2 )
wl_by_day['wl_min'] = np.maximum(wl_by_day.wl_low_1, wl_by_day.wl_low_2 )
wl_by_day = wl_by_day[['wl_max','wl_min']]
wl_by_day.head()

,wl_max,wl_min
date_time,,
2000-01-01,5.043,1.546
2000-01-02,4.974,1.644
2000-01-03,5.234,1.651
2000-01-04,5.270,1.549
2000-01-05,5.617,1.792


In [8]:
dist = pkg.reference('sd_river_discharge').read_csv(parse_dates=True).set_index('datetime').resample('1d').mean()
dist = dist[['discharge_rate']].copy()
dist.head()

,discharge_rate
datetime,
1988-01-01,22.0
1988-01-02,20.0
1988-01-03,18.0
1988-01-04,18.0
1988-01-05,21.0


In [9]:
usecols = ['date', 'reporttpye', 'dailyprecip', 'hourlyprecip']
lcd = pkg.reference('local_climate').read_csv(parse_dates=True, usecols=usecols) #.set_index('datetime').resample('1d').mean()

lcd.rename(columns={'reporttpye': 'reporttype'}, inplace=True)
lcd = lcd[lcd.reporttype=='FM-15'].copy()
lcd.fillna(0, inplace=True)
lcd['date'] = lcd.date.dt.date
lcd = lcd.groupby('date').dailyprecip.max()


In [10]:
comb = dist.join(lcd).join(wl_by_day)
comb.head()

,discharge_rate,dailyprecip,wl_max,wl_min
datetime,,,,
1988-01-01,22.0,NaN,NaN,NaN
1988-01-02,20.0,NaN,NaN,NaN
1988-01-03,18.0,NaN,NaN,NaN
1988-01-04,18.0,NaN,NaN,NaN
1988-01-05,21.0,NaN,NaN,NaN


In [11]:
comb.corr()

,discharge_rate,dailyprecip,wl_max,wl_min
discharge_rate,1.000000,0.637036,0.045262,-0.001102
dailyprecip,0.637036,1.000000,0.050992,0.012542
wl_max,0.045262,0.050992,1.000000,-0.155287
wl_min,-0.001102,0.012542,-0.155287,1.000000


In [12]:
%mt_materialize comb '/Users/eric/Library/Application Support/metapack/_materialized_data/sandiegodata.org-water_quality-3' 

{
    "df_name": "comb",
    "path": "/Users/eric/Library/Application Support/metapack/_materialized_data/sandiegodata.org-water_quality-3/comb.csv"
}


In [13]:

%mt_materialize_all '/Users/eric/Library/Application Support/metapack/_materialized_data/sandiegodata.org-water_quality-3'


[]


In [14]:

%mt_show_metatab



Declare: metatab-latest
Section: Resources


In [15]:

%mt_show_libdirs



[]
